In [ ]:
# Set up path for importing modules
import sys, os
sys.path.insert(0, os.path.abspath(".."))
from init_env import set_project_root
project_root = set_project_root()

# Imports
import numpy as np
import algorithmsML as algorithms
import functionsML as functions
from optSolverML import optSolver

import scipy.io

In [19]:
# Define three class objects required in 'optSolver' and their attributes 

class Problem:
    def __init__(self, name, w0, X_train, y_train, X_test, y_test):
        self.name = name
        self.w0 = w0
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        match name:
            case "LinearLeastSquares":
                self.compute_f = functions.linear_least_squares_func
                self.compute_g = functions.linear_least_squares_grad
                self.compute_f_g = functions.linear_least_squares_func_grad
                self.predict = functions.linear_least_squares_predict
            case "LogisticRegression": 
                # Lambda function so I only need to pass x
                self.compute_f = functions.logistic_regression_func
                self.compute_g = functions.logistic_regression_grad
                self.compute_f_g = functions.logistic_regression_func_grad
                self.predict = functions.logistic_regression_predict
            case _:
                raise ValueError("problem not defined!!!")
            
class Method:
    def __init__(self, name, **options):
        self.name = name
        self.options = options
        
class Options:
    def __init__(self,term_tol=1e-6, max_iterations=1e2):
        self.term_tol = term_tol
        self.max_iterations = max_iterations  

# Australian Analysis

In [ ]:
# Load the data
australian = scipy.io.loadmat(project_root / 'hw4/data/australian.mat')

# Mushroom Analysis

In [ ]:
# Load the data
mushroom = scipy.io.loadmat(project_root / "hw4/data/mushroom.mat")

X_train = australian['X_train']
X_test = australian['X_test']

y_train = australian['y_train']
y_test = australian['y_test']

## Linear Least Squares

In [ ]:
problem = Problem('LinearLeastSquares', w0=np.zeros((X_train.shape[1], 1)), 
                  X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
method = Method('GradientDescent', step_type='Constant', constant_step_size=1e-3)
options = Options(term_tol=1e-6, max_iterations=1e3)